In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
import random
import json
from datetime import datetime
from genetic_algo import GeneticAlgorithm
from training_and_evaluation import train_model , evaluate
from augmentation import get_transforms
from Dataset import PneumoniaDataset

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [2]:
def main():
    """Main execution function"""
    
    # Configuration
    DATA_PATH = 'chest_xray'  # Update with your data path
    POPULATION_SIZE = 8
    GENERATIONS = 3
    EPOCHS_PER_INDIVIDUAL = 5
    FINAL_TRAINING_EPOCHS = 20
    
    # Check if data path exists
    if not os.path.exists(DATA_PATH):
        print(f"Error: Data path '{DATA_PATH}' not found!")
        print("Please download the Chest X-Ray dataset from:")
        print("https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia")
        print("\nExpected structure:")
        print("chest_xray/")
        print("  ├── train/")
        print("  │   ├── NORMAL/")
        print("  │   └── PNEUMONIA/")
        print("  ├── val/")
        print("  │   ├── NORMAL/")
        print("  │   └── PNEUMONIA/")
        print("  └── test/")
        print("      ├── NORMAL/")
        print("      └── PNEUMONIA/")
        return
    
    # Run genetic algorithm
    ga = GeneticAlgorithm(
        data_path=DATA_PATH,
        population_size=POPULATION_SIZE,
        generations=GENERATIONS,
        epochs_per_individual=EPOCHS_PER_INDIVIDUAL
    )
    
    best_hyperparams, best_fitness = ga.evolve()
    
    # Save optimization history
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    history_file = f'ga_history_{timestamp}.json'
    with open(history_file, 'w') as f:
        json.dump({
            'best_hyperparameters': best_hyperparams,
            'best_fitness': best_fitness,
            'history': ga.history
        }, f, indent=2)
    print(f"Optimization history saved to {history_file}")
    
    # Train final model with best hyperparameters
    print(f"\n{'='*80}")
    print("Training Final Model with Best Hyperparameters")
    print(f"{'='*80}\n")
    
    final_model, val_acc, f1 = train_model(best_hyperparams, DATA_PATH, 
                                          epochs=FINAL_TRAINING_EPOCHS)
    
    # Evaluate on test set
    _, test_transform = get_transforms(best_hyperparams['augmentation'])
    test_dataset = PneumoniaDataset(DATA_PATH, 'test', transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=best_hyperparams['batch_size'],
                           shuffle=False, num_workers=2)
    
    criterion = nn.CrossEntropyLoss()
    test_loss, test_acc, test_precision, test_recall, test_f1 = \
        evaluate(final_model, test_loader, criterion, device)
    
    print(f"\n{'='*80}")
    print("Final Test Results:")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Test Precision: {test_precision:.4f}")
    print(f"  Test Recall: {test_recall:.4f}")
    print(f"  Test F1 Score: {test_f1:.4f}")
    print(f"{'='*80}\n")
    
    # Save final model
    model_file = f'pneumonia_cnn_best_{timestamp}.pth'
    torch.save({
        'model_state_dict': final_model.state_dict(),
        'hyperparameters': best_hyperparams,
        'test_accuracy': test_acc,
        'test_f1': test_f1
    }, model_file)
    print(f"Final model saved to {model_file}")


if __name__ == '__main__':
    main()


Starting Genetic Algorithm Optimization
Population Size: 8, Generations: 3


Generation 1/3

Evaluating individual:
  LR: 0.003752, Batch: 64, Filters: 16, Dropout: 0.670
train set: 5216 images (3875 pneumonia, 1341 normal)
val set: 16 images (8 pneumonia, 8 normal)
Epoch 1/5: Train Loss: 1.2927, Train Acc: 73.26% | Val Loss: 1.4641, Val Acc: 50.00%, F1: 0.6667
Epoch 2/5: Train Loss: 0.3587, Train Acc: 82.75% | Val Loss: 1.0484, Val Acc: 62.50%, F1: 0.7273
Epoch 3/5: Train Loss: 0.3537, Train Acc: 84.68% | Val Loss: 0.8148, Val Acc: 62.50%, F1: 0.7273
Epoch 4/5: Train Loss: 0.3182, Train Acc: 87.10% | Val Loss: 2.0656, Val Acc: 50.00%, F1: 0.6667
Epoch 5/5: Train Loss: 0.3197, Train Acc: 87.27% | Val Loss: 1.6258, Val Acc: 56.25%, F1: 0.6957
  Fitness: 64.62 (Val Acc: 62.50%, F1: 0.6957)

Evaluating individual:
  LR: 0.005991, Batch: 16, Filters: 64, Dropout: 0.194
train set: 5216 images (3875 pneumonia, 1341 normal)
val set: 16 images (8 pneumonia, 8 normal)
Epoch 1/5: Train Loss: 2.